In [21]:
import numpy as np

path = '../Data/'

f = open(path + '/DatasetA_train_20180813/attributes_per_class.txt')
data_per_class = f.readlines()
 
data = []
for line in data_per_class:
    data.append(line.split())
print(data)

head = list(['lable',
    'animal',
'transportation',
'clothes',
'plant',
'tableware',
'device',
'black',
'white',
'blue',
'brown',
'orange',
'red',
'green',
'yellow',
'has_feathers',
'has_four_legs',
'has_two_legs',
'has_two_arms',
'for_entertainment',
'for_business',
'for_communication',
'for_family',
'for_office_use',
'for_personal',
'gorgeous',
'simple',
'elegant',
'cute',
'pure',
'naive'])

[['ZJL1', '1', '0', '0', '0', '0', '0', '0.5', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['ZJL10', '1', '0', '0', '0', '0', '0', '0.5', '0', '0', '0.7', '0', '0', '0', '0.2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['ZJL100', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0'], ['ZJL101', '0', '0', '0', '0', '0', '0', '0.8', '0.1', '0.3', '0.1', '0.3', '0.1', '0.3', '0.3', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0'], ['ZJL102', '0', '0', '0', '0', '0', '0', '0', '0.1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0.5', '0', '0', '0', '0', '0', '0', '0'], ['ZJL103', '0', '0', '0', '0', '0', '0', '0', '0.5', '0.5', '0.5', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0

In [22]:
import pandas as pd
data = pd.DataFrame(data,columns= head)
data = data.set_index('lable')
 
# data_1 = data.loc[:,'animal':'device']
# data_2 = data.loc[:,'black':'yellow']
# data_3 = data.loc[:,'has_feathers':'has_two_arms']
# data_4 = data.loc[:,'for_entertainment':'for_personal']
 
# data_1 = (data_1 >= '1') & 1
# data_1['non_1'] = 1 - data_1.sum(axis = 1)
 
# #颜色属性不太好用（比如car\pan\cost），这里就不做复杂处理，模型训练时暂不用
# data_2 = (data_2 > '0.7') & 1
# data_2.sum(axis = 1)
 
# data_3 = (data_3 >= '1') & 1
# data_3.sum(axis = 1)
 
# data_4 = (data_4 >= '1') & 1
# data_4.sum(axis = 1)
 
# data_new = pd.DataFrame(data['lable'])
# data_new = pd.DataFrame(np.hstack((data_new, data_1, data_2, data_3, data_4)))
# data_new.to_csv("test_1.csv", index=None)
# print("data saved to test_1.csv.")

In [3]:
data_x = data.loc[:,'animal':'device']
# (data_x >= '1') & 1

In [30]:
#build model
from keras import Model, Sequential
from keras.applications import VGG16, VGG19
from keras.layers import Flatten, Dense
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras import backend as K
K.set_image_data_format('channels_last')


base_model = VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
 
# classes = {
#     'cla' : 6+1,
#   #  'clo' : 8, #暂且不用
#     'has' : 4,
#     'for' : 6
#   #  'is' : 6 #暂且不用
# }
 
# x = base_model.output
# x = Flatten()(x)
# x = Dense(1024, activation='relu')(x)
# # predictions = [Dense(n, activation='softmax', name=m)(x) for m,n in classes.items()]
# predictions = [Dense(1, activation='sigmoid', name=m.replace(' ', '_'))(x) for m in head[1:]]
 
# model = Model(inputs=base_model.input, outputs= predictions)
# model.compile(optimizer='rmsprop', loss='binary_crossentropy')
# model.summary()

58892288/58889256 [==============================] - 15s 0us/step


In [33]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [97]:
from keras.layers import *

attr_input = Input(shape = (30,))
word_emb = Input(shape = (300,))
imag_classifier = Input(shape = (1024,))

attr_dense = Dense(2048, activation="relu")(attr_input)
# attr_dense = Dense(2048, activation="relu")(attr_input)
# attr_dense = Dense(1024, activation="relu")(attr_input)
# attr_dense = Dense(512, activation="relu")(attr_dense)

attr_word_emb = Concatenate()([word_emb, attr_dense])

attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb)
attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)
attr_word_emb_dense = Dense(1024, activation="relu")(attr_word_emb_dense)
attr_word_emb_dense = Dense(512, activation="relu")(attr_word_emb_dense)
attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)

# mse = Dense(6, activation="softmax")(dense_output)

model = Model([attr_input, word_emb], attr_word_emb_dense)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 2048)         63488       input_21[0][0]                   
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 2348)         0           input_22[0][0]                   
                                                                 dense_25[0][0]                   
__________

In [66]:
class_emb = pd.read_csv(path + '/DatasetA_train_20180813/class_wordembeddings.txt', index_col = 0, sep = ' ', header = None)
class_emb = class_emb.apply(lambda s: np.array([float(x) for x in s]), axis = 1)
class_id_to_name = pd.read_csv(path + '/DatasetA_train_20180813/label_list.txt', 
                               index_col = 0, sep = '\t', header = None)[1]

In [67]:
class_id_to_name.head()

0
ZJL1       goldfish
ZJL10     tarantula
ZJL100    drumstick
ZJL101     dumbbell
ZJL102     flagpole
Name: 1, dtype: object

In [83]:
import pandas as pd
import numpy as np
# data_atten = pd.read_csv(r'test_1.csv')
# data_atten = data_atten.set_index('0')
data_train = open(path + '/DatasetA_train_20180813/train.txt')
data_train = data_train.readlines() #[:2000]#!!!!!!!!!!!!!!!!!!!!!！！！！！！！！！！！！！！！！！！这里要注意修改
imag_path = path + r'/DatasetA_train_20180813/train/'
length = len(data_train)
print ('Train length: ', length)
# train_x = np.zeros((length, 64, 64, 3))
train_attr = np.zeros((length, 30))
train_emb = np.zeros((length, 300))
# train_y = np.zeros((length, 1024))
train_image = np.zeros((length, 64, 64, 3))
for i in range(length):
    image_id, class_id = data_train[i].split()
    img = image.load_img(imag_path + image_id)
    train_attr[i] = data.loc[class_id]
    train_emb[i] = class_emb.loc[class_id_to_name[class_id]]
    train_image[i] = image.img_to_array(img)
print ('-----Image Features Predict-----')
train_y = np.reshape(base_model.predict(train_image, batch_size = 256, verbose = 1), (length, -1)) #data.loc[class_id] #data_atten.loc[n]

Train length:  38221
-----Image Features Predict-----
38221/38221 [==============================] - 2618s 68ms/step


In [100]:
import pickle

with open(path + 'train_vgg16.pickle', 'wb+') as handle:
    pickle.dump(train_y, handle)

In [98]:
    
    
#np.save('abc.npy', train_x)
#train_x = np.load('abc.npy')
# h = model.fit(train_x, {'cla':train_y[:,:7], 'has':train_y[:,15:19], 'for':train_y[:,19:25]},
#               epochs=50,validation_split=0.2, batch_size = 128, shuffle=True)
h = model.fit([train_attr, train_emb], np.log1p(train_y),
              epochs=50,validation_split=0.2, batch_size = 64, shuffle=True, verbose = 1)

Train on 30576 samples, validate on 7645 samples
Epoch 1/50
30576/30576 [==============================] - 151s 5ms/step - loss: 1.1457 - val_loss: 1.1482
Epoch 2/50
30576/30576 [==============================] - 156s 5ms/step - loss: 1.0889 - val_loss: 1.1236
Epoch 3/50
 6016/30576 [====>.........................] - ETA: 2:01 - loss: 1.0714

KeyboardInterrupt: 

In [19]:
np.ndenumerate(data.columns.values)

In [ ]:
 
data_test = open("D:\\data\\pic\\DatasetA_test_20180813\\DatasetA_test\\image.txt")
data_test = data_test.readlines()
path_test = r'D:\data\pic\DatasetA_test_20180813\DatasetA_test\test\\'
length = len(data_test)
data_test_x = np.zeros((length, 64, 64, 3))
for i in range(length):
    m = data_test[i][:-1]
    img = image.load_img(path_test + m)
    data_test_x[i] = image.img_to_array(img)
 
res = model.predict(data_test_x)
 
#欧氏距离：
data_atten_res = []
for i in range(160,241):
    key = 'ZJL' + str(i)
    data_atten_res.append(np.hstack((data_atten.loc[key][1:8], data_atten.loc[key][16:20], data_atten.loc[key][20:26])))
 
vec_result = []#np.zores((len(res[0]), 18))
for i in range(len(res[0])):
    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i])))
dis_list = []
 
for j in range(len(data_atten_res)):
    vec1 = vec_result[0]
    vec2 = data_atten_res[j]
    dist = np.linalg.norm(vec1 - vec2)
    dis_list.append(dist)
print("witch is : "+ str(dis_list.index(min(dis_list))) + " and min dis = " + str(min(dis_list)))